# Diplomatura en Python
___

## TP Nivel Intermedio


### Alumno: Edio José Guizzo

### Aplicación Desarrollada

En este trabajo se explica el codigo desarrollado para una aplicación de registro de gastos. A continuación se puede observar una imagen de la misma:

![img1](app.png "Captura de pantalla")

El codigo se encuentra dividido en cuatro modulos. Cumpliendo cada uno un rol particular, e integrandose para permitir la ejecución del programa. El primero de ellos corresponde al *controlador*, otro es el *modelo*, luego estan las *vistas* de la ventana principal y secundaria, y el ultimo contiene la logica de validación **REGEX.** Tal como se aprecia en la separación de los modulos el script sigue el modelo de desarrollo MVC, lo cual para aplicaciones de este tipo resulta mas practico y ordenado. Facilitando la comprensión del codigo a otros programadores.


# MODULOS - MVC

# controlador.py

Este modulo es el principal y permite la ejecución del resto del script. Es por esto que para ejecurtalo si o si se debe hacer directamente desde el archivo controlador.py y no de otra manera. Primeramente el codigo crea una ventana de **tkinter** llamada root, y su correspondiente mainloop() para cerrar el bucle. A su vez se crea una instancia de la **class Controlador():** a la que se le pasa la como elemento la ventana para que pueda inicializar el resto del programa. 


La clase **Controlador** cuenta con tres metodos, un constructor **def __init__(self, root):**, una función que ejecuta la ventana principal **def ventana_principal(self,):**, y por ultimo otra función que ejecuta la ventana secundaria **def ventana_secundaria(self, datos, objeto):**

In [ ]:
from tkinter import Tk
from vista import *
from vista_2 import *


class Controlador():

    def __init__(self, root):
        self.root_controlador = root
        self.ventana_principal()

    def ventana_principal(self,):
        Ventana_principal(self.root_controlador)

    def ventana_secundaria(self, datos, objeto):
        print("Despliego ventana secundaria")
        self.root_sec = Toplevel()
        Ventana_secundaria(self.root_sec, datos, objeto)
        self.root_sec.grab_set()
        self.root_sec.focus_set()
        self.root_sec.wait_window()


if __name__ == "__main__":
    root = Tk()
    Controlador(root)
    root.mainloop()

# modelo.py

Aqui se encuentra toda la logica de funcionamiento de la aplicación, este modulo se encarga de interacturar con la vista y el controlador. 

Primeramente se importan los modulos y librerias necesarios para la ejecución del codigo. Posteriormente se define la clase principal llamada **class Abmc():** en ella se encuentran los metodos que permiten la interacción vista-modelo y tambien otros propios del modelo.

In [ ]:
from tkinter import messagebox
import sqlite3
from controlador import *
from regex import validador


class Abmc():
    


### def __init__(self,):

Este metodo es el **constructor**, en el se inicializa la conexión con la base datos y luego se crea la tabla necesaria para el registro en caso de que no se haya creado antes. La tabla posee los siguientes campos:


* **id**
    * entero, autoincremental, clave primaria
* **concepto**
    * texto, no nulo.
* **valor**
    * entero.
* **fecha**
    * texto, no nulo.
* **descripcion**
    * texto, no nulo.

In [ ]:
    def __init__(self,):
        self.data_base = 'registro.db'
        try:
            db = self.conexion()
            cursor = db.cursor()
            cursor.execute("""CREATE TABLE IF NOT EXISTS gastos(
                id integer PRIMARY KEY AUTOINCREMENT,
                concepto TEXT NOT NULL,
                valor INTEGER,
                fecha TEXT NOT NULL,
                descripcion TEXT NOT NULL)""")
            db.commit()
        except sqlite3.DatabaseError as mensaje:
            self.pop_up_error("Error en DB", mensaje)
        else:
            print("Base de datos creada con éxito")
        finally:
            db.close()

### def conexion(self,):

Establece la conexión con la base de datos.

In [ ]:
    def conexion(self,):
        db = sqlite3.connect(self.data_base)
        return db

### def info_db_treeview(self, tree):

Mediante el uso de la *query* **SELECT** se asignan al cursor todas las filas de la base de datos. Luego se pasan a una variable empleando la función **cursor.fetchall()**. Esta función nos retorna una lista de filas, para recorrerla se usa un bucle **for** para recorrer todas las fila, imprimiendo los campos de la lista en las columnas de la tabla del treeview mediante **tree.insert()**.


In [ ]:
    def info_db_treeview(self, tree):
        tree.delete(*tree.get_children())
        try:
            db = self.conexion()
            cursor = db.cursor()
            cursor.execute("SELECT * FROM gastos")
            db.commit()
            registros = cursor.fetchall()
        except sqlite3.DatabaseError as mensaje:
            self.pop_up_error("Error en DB", mensaje)
            print("Error en base de datos")
        else:
            for info in registros:
                tree.insert("", "end", text=info[0], values=(
                    info[1], info[2], info[3], info[4]))
        finally:
            db.close()

### def alta_db(self, tree, *args):

Esta función es ejecutada cuando se presiona el boton **Cargar**. Primero valida los campos ingresados y luego se conecta con la base de datos. Esta secuencia se encuentra dentro de un **try**, donde posteriormente se inserta en la base de datos los valores retornados por la función **validador()**.

Una vez realizada el alta, *cursor.execute(sql_query, valores)*, se confirma la misma con un **db.commit()** para luego cerrar la conexión con la base de datos.

Posteriormente si todo fue correcto y sin errores se actualiza la vista de la tabla del treeview, llamando a la función **info_db_treeview()** y se limpian las celdas de texto con la función **limpiar_celdas()**. 

En caso de originarse un error en el proceso anterior, existen dos condiciones **except**, una para errores de **sqlite3** y otra para error en el **validador**.

In [ ]:
    def alta_db(self, tree, *args):
        sql_query = "INSERT INTO gastos('concepto', 'valor', 'fecha', 'descripcion') VALUES(?,?,?,?)"
        try:
            valores = validador(*args)
            db = self.conexion()
            cursor = db.cursor()
            print("===============================")
            print("Datos ingresados: ")
            for n in valores:
                print(n, end=" ")
            cursor.execute(sql_query, valores)
            db.commit()
        except sqlite3.DatabaseError as mensaje:
            self.pop_up_error("Error en DB", mensaje)
            print("No se pudo cargar")
        except TypeError:
            self.pop_up_error("Error", valores)
        else:
            self.info_db_treeview(tree)
            self.limpiar_celdas(*args)
            print("Carga exitosa")
        finally:
            db.close()

### def baja_db(self, item_id):

Primeramente nos conectamos a la base de datos, para luego tomar el **ID** del item a borrar. Con ese dato enviamos la *query* a la base de datos para que elimine la fila correspondiente a ese numero de *id*.

In [ ]:
    def baja_db(self, item_id):
        try:
            db = self.conexion()
            cursor = db.cursor()
            cursor.execute("DELETE FROM gastos WHERE id = ?", (item_id,))
            db.commit()
        except sqlite3.DatabaseError as mensaje:
            self.pop_up_error("Error en DB", mensaje)
            print("No se pudo borrar")
        else:
            print("Borrado exitoso")
        finally:
            db.close()

### def limpiar_celdas():

La funcion se encarga de limpiar las celdas de ingreso de texto, esto lo realizamos con el metodo **.delete()**.

In [ ]:
    def limpiar_celdas(self, *args):
        for valor in args:
            valor.delete(0, END)

### def borrar_registro_treeview():

Aqui mediante el uso de **tkinter** y su función **focus()**, podemos selececcionar directamente una fila de la tabla de treeview. Esta nos devuelta la información de toda la fila, de la que tomamos el valor de la **key** *'text'*, en donde se encuentra el **ID** asignado por al base de datos.

Posteriormente se llama a la función **tree.delete**, la cual borra la fila de la tabla del treeview, de acuerdo al *item_id* que le pasa la función. Lo mismo sucede con **baja_db** que elimina la fila de la base de datos.


En caso de no haber seleccionado un item y apretar el botón **Borrar**, se despliega un *pop-up* con la leyenda que *Seleccione el item a eliminar*.

In [ ]:
    def borrar_registro_treeview(self, tree):
        item_focus = tree.focus()
        item_actual = tree.item(item_focus)
        if item_actual['text'] != '':
            item_id = item_actual['text']
            print("===============================")
            print("Se borró el item: ", item_id)
            tree.delete(item_focus)
            self.baja_db(item_id)  # Llamo funcion que borra registro en DB.
        else:
            self.pop_up_error("Error al borrar", "Seleccione item a eliminar")

## def buscar_registro(self, tree, objeto_base):

Mediante el uso de **tkinter** y su función **focus()**, podemos selececcionar directamente una fila de la tabla de treeview. Esta nos devuelta la información de toda la fila, de la que tomamos el valor de la **key** *'text'*, en donde se encuentra el **ID** asignado por al base de datos. Dicho valor se lo pasamos a la función **self.datos_busqueda(item_id)**, la cual nos despliega una nueva ventana donde muestra el detalle del item seleccionado.

En caso de no haber seleccionado un item y apretar el botón **Buscar**, se despliega un *pop-up* con la leyenda que *Seleccione el item a buscar*.

In [ ]:
    def buscar_registro(self, tree, objeto_base):
        self.objeto_base = objeto_base
        self.tree = tree
        item_focus = self.tree.focus()
        item_actual = self.tree.item(item_focus)
        if item_actual['text'] != '':
            item_id = item_actual['text']
            print("===============================")
            print("Seleccionó item: ", item_id)
            # Llamo funcion que busca registro en DB.
            self.datos_busqueda(item_id)
        else:
            self.pop_up_error("Error al buscar", "Seleccione item a buscar")
            messagebox.showinfo(
                message="Seleccione item a buscar", title="Error al modificar")

### def datos_busqueda(self, item_id):

Aqui se recibe el **item_id** enviado por la función anterior. Empleando el *cursor*, seleccionamos de la base datos la fila correspondiente al id elegido. Mediante **cursor.fetchone()** se obtienen los datos de esa fila, para luego de definir toda la ventana, se despliegue en la misma la información del id elegido. Estos campos de textos mostrados, pueden ser modificados y luego actualizados en la base de datos mediante el boton **Modificar**.

In [ ]:
    def datos_busqueda(self, item_id):
        try:
            db = self.conexion()
            cursor = db.cursor()
            cursor.execute("SELECT * FROM gastos WHERE id = ?", (item_id,))
            self.datos = cursor.fetchone()
        except sqlite3.DatabaseError as mensaje:
            self.pop_up_error("Error en DB", mensaje)
        else:
            self.call_ventana(self.objeto_base)
        finally:
            db.close()

### def modificar_registro(self, id, *args):

Se reciben los valores de los campos a modificar. Luego los valido a traves de la funcion encargada de ello. Posteriormente mediante la *SQL Query* **UPDATE** actualizo estos campos en donde se encuentre el **ID** correspondiente.

In [ ]:
    def modificar_registro(self, id, *args):
        datos = validador(*args)
        datos.append(id)
        print(datos)
        sql_query = "UPDATE gastos SET concepto=?, valor=?, fecha=?, descripcion=? WHERE id = ?"
        """datos = (concepto2.get(), valor2.get(),
                 fecha2.get(), descripcion2.get(), id)
        """
        try:
            db = self.conexion()
            cursor = db.cursor()
            cursor.execute(sql_query, datos)
            db.commit()
            self.info_db_treeview(self.tree)
            # self.limpiar_celdas()
        except sqlite3.DatabaseError as mensaje:
            self.pop_up_error("Error en DB", mensaje)
        else:
            print("Modificacion exitosa")
        finally:
            db.close()

###    def call_ventana(self, objeto_base):

Permite inicializar la *ventana_secundaria* a traves de una instancia al metodo correspondiente de la clase **Controlador**.

In [ ]:
    def call_ventana(self, objeto_base):
        Controlador.ventana_secundaria(self, self.datos, objeto_base)

### def pop_up_error(self, *args):

Este método recibe el titulo y texto del error, para luego desplagarlo en una ventana del tipo **messagebox.**

In [ ]:
    def pop_up_error(self, *args):
        messagebox.showinfo(message=args[1], title=args[0])

# vista.py

Todo lo referido a la interfaz gráfica de la ventana principal. Aca se incluye los parametros de los botones, labels, entradas de texto, treeview, y las funciones que interactuan con el modelo.

In [ ]:
from tkinter import ttk
from tkinter import *
import datetime
from modelo import Abmc

# Defino el formato en el que obtengo la fecha actual.
fecha_actual = datetime.datetime.now().strftime("%d/%m/%Y")


class Ventana_principal():

    def __init__(self, window):
        self.root = window

        self.concepto = StringVar()
        self.valor = StringVar()
        self.fecha = StringVar(value=fecha_actual)
        self.descripcion = StringVar()
        self.ancho = 40
        self.objeto_base = Abmc()

        # Frame
        self.root.title("Registro de Gastos")
        self.root.resizable(width=False, height=False)
        self.treeview = ttk.Treeview(self.root)

        # Etiquetas
        self.label0 = Label(self.root, text="Ingrese los valores",
                            background='black', foreground='white')
        self.label0.grid(row=0, column=0, columnspan=6,
                         padx=5, pady=5, sticky=W + E)

        self.label1 = Label(self.root, text="Concepto:")
        self.label1.grid(row=1, column=0, sticky=W, padx=5, pady=5)

        self.label2 = Label(self.root, text="Valor en $:")
        self.label2.grid(row=2, column=0, sticky=W, padx=5, pady=5)

        self.label3 = Label(self.root, text="(no usar $ o .)")
        self.label3.grid(row=2, column=1, sticky=E, padx=5, pady=5)

        self.label4 = Label(self.root, text="Fecha:")
        self.label4.grid(row=3, column=0, sticky=W, padx=5, pady=5)

        self.label5 = Label(self.root, text="(dd/mm/yyyy)")
        self.label5.grid(row=3, column=1, sticky=W+E, padx=5, pady=5)

        self.label6 = Label(self.root, text="Descripción:")
        self.label6.grid(row=4, column=0, sticky=W, padx=5, pady=5)

        self.label7 = Label(self.root, text="Registro")
        self.label7.grid(row=5, column=0, sticky=W, padx=5, pady=15)

        # Entradas
        self.concepto_entry = Entry(
            self.root, width=self.ancho, textvariable=self.concepto, bg='white')
        self.concepto_entry.grid(row=1, column=1, padx=5, pady=5, sticky=W)

        self.valor_entry = Entry(
            self.root, width=15, textvariable=self.valor, bg='white')
        self.valor_entry.grid(row=2, column=1, padx=5, pady=5, sticky=W)

        self.fecha_entry = Entry(
            self.root, width=10, textvariable=self.fecha, bg='white')
        self.fecha_entry.grid(row=3, column=1, padx=5, pady=5, sticky=W)

        self.descripcion_entry = Entry(self.root, width=self.ancho,
                                       textvariable=self.descripcion, bg='white')
        self.descripcion_entry.grid(row=4, column=1, padx=5, pady=5, sticky=W)

        # Botones
        self.ancho_boton = 7
        self.b_alta = Button(self.root, text="Cargar", width=self.ancho_boton,
                             command=lambda: self.alta())
        self.b_alta.grid(row=5, column=1, padx=5,
                         pady=5, sticky=E, columnspan=2)

        self.b_baja = Button(self.root, text="Borrar", width=self.ancho_boton,
                             command=lambda: self.borrar())
        self.b_baja.grid(row=8, column=2, padx=5, pady=5)

        self.b_busqueda = Button(self.root, text="Buscar",
                                 width=self.ancho_boton, command=lambda: self.buscar())
        self.b_busqueda.grid(row=8, column=1, padx=5, pady=5, sticky='E')

        self.b_salir = Button(self.root, text="Salir", width=self.ancho_boton,
                              command=lambda: (self.root.destroy(), print("Bye !")))
        self.b_salir.grid(row=8, column=0, padx=5, pady=5, sticky='W')

        # Treeview
        self.tree = ttk.Treeview(self.root)
        self.tree.grid(column=0, row=6, columnspan=5)
        scrollbar = ttk.Scrollbar(
            self.root, orient="vertical", command=self.tree.yview)
        scrollbar.grid(row=6, column=5, sticky='nse')
        self.tree.configure(yscrollcommand=scrollbar.set)
        self.tree["columns"] = ("col1", "col2", "col3", "col4")
        self.tree.column("#0", width=50, minwidth=50, anchor=W)
        self.tree.heading('#0', text='ID')
        self.tree.column("col1", width=100, minwidth=80)
        self.tree.heading('col1', text='Concepto')
        self.tree.column("col2", width=80, minwidth=80)
        self.tree.heading('col2', text='Valor')
        self.tree.column("col3", width=80, minwidth=80)
        self.tree.heading('col3', text='Fecha')
        self.tree.column("col4", width=180, minwidth=100)
        self.tree.heading('col4', text='Descripción')
        # Actualizo treeview luego de crearlo
        self.objeto_base.info_db_treeview(self.tree)

    def alta(self,):
        self.objeto_base.alta_db(self.tree, self.concepto_entry, self.valor_entry,
                                 self.fecha_entry, self.descripcion_entry)

    def borrar(self,):
        self.objeto_base.borrar_registro_treeview(self.tree)

    def buscar(self,):
        self.objeto_base.buscar_registro(self.tree, self.objeto_base)

    def modificar_registro(self, id, concepto2, valor2, fecha2, descripcion2):
        self.objeto_base(id, concepto2, valor2, fecha2, descripcion2)

# vista_2.py

Todo lo referido a la interfaz gráfica de la ventana secundaria. Incluyendo los parametros de los botones, labels, entradas de texto, y las funciones que interactuan con el modelo.

In [ ]:
from tkinter import *

class Ventana_secundaria():

    def __init__(self, window, datos, objeto):
        self.root_sec = window
        self.objeto_base = objeto

        self.id = datos[0]
        self.concepto2 = StringVar(value=datos[1])
        self.valor2 = StringVar(value=datos[2])
        self.fecha2 = StringVar(value=datos[3])
        self.descripcion2 = StringVar(value=datos[4])

        # Frame
        self.root_sec.title("Búsqueda de datos")
        self.root_sec.resizable(width=False, height=False)

        # Etiquetas
        self.label0 = Label(self.root_sec, text="ID:")
        self.label0.grid(row=1, column=0, sticky=E, padx=5, pady=5)

        self.label1 = Label(
            self.root_sec, text=self.id, width=4, bg='white')
        self.label1.grid(row=1, column=1, padx=5, pady=5, sticky=W)

        self.label2 = Label(self.root_sec, text="Concepto:")
        self.label2.grid(row=2, column=0, sticky=E, padx=5, pady=5)

        self.label3 = Label(self.root_sec, text="Valor en $:")
        self.label3.grid(row=3, column=0, sticky=E, padx=5, pady=5)

        self.label4 = Label(self.root_sec, text="Fecha:")
        self.label4.grid(row=4, column=0, sticky=E, padx=5, pady=5)

        self.label5 = Label(self.root_sec, text="(dd/mm/yyyy)")
        self.label5.grid(row=4, column=1, sticky=W+E, padx=5, pady=5)

        self.label6 = Label(self.root_sec, text="Descripción:")
        self.label6.grid(row=5, column=0, sticky=E, padx=5, pady=5)

        # Entradas
        self.ancho = 40

        self.concepto_entry = Entry(self.root_sec, width=self.ancho,
                                    textvariable=self.concepto2, bg='white')
        self.concepto_entry.grid(row=2, column=1, padx=5, pady=5)

        self.valor_entry = Entry(
            self.root_sec, width=15, textvariable=self.valor2, bg='white')
        self.valor_entry.grid(row=3, column=1, padx=5, pady=5, sticky=W)

        self.fecha_entry = Entry(
            self.root_sec, width=10, textvariable=self.fecha2, bg='white')
        self.fecha_entry.grid(row=4, column=1, padx=5, pady=5, sticky=W)

        self.descripcion_entry = Entry(self.root_sec, width=self.ancho,
                                       textvariable=self.descripcion2, bg='white')
        self.descripcion_entry.grid(row=5, column=1, padx=5, pady=5)

        # Botones
        b_alta = Button(self.root_sec, text="Modificar", width=7,
                        command=lambda: self.modificar_registro())
        b_alta.grid(row=6, column=1, padx=5, pady=5, sticky='E')

    def modificar_registro(self,):
        self.objeto_base.modificar_registro(
            self.id, self.concepto_entry, self.valor_entry, self.fecha_entry, self.descripcion_entry)
        print("===============================")
        print("Cierro ventana secundaria")
        self.root_sec.destroy()


# regex.py

Este modulo contiene el codigo correspondiente a la validación **regex**. Cuenta con una función llamada validador, la cual es invocada desde el modelo cada vez que se requiere verificar un campo de entrada.

### def validador(concepto, valor, fecha, descripcion):

Mediante el metodo **re.compile()** se define el patrón *REGEX* para los distintos campos. Cada patrón se encuentra definido en una lista llamada **patrones**. A su vez con un ciclo **for** se genera una lista llamada **valores** donde se almacena cada entrada. 


Posteriormente mediante el uso de otro ciclo **for** se procede a validar cada campo con su patrón, en caso de no ser valido se genera un error del tipo **TypeError** con un mensaje informativo. En caso de existir error, este es detectado por un **try/except/else** el cual retorna el mensaje a la función inicial desde donde se invocó.


La condición **regex** empleada permite el ingreso de caracteres alfanumericos para el *Concepto* y la *Descripción*. En el caso del *Precio* unicamente permite numeros y por ultimo para la *Fecha* limita el acceso de acuerdo el siguiente formato *dd/mm/yyyy*.

Si la validación es correcta, la función retorna una lista con los campos validados.

In [ ]:
import re

def validador(*args):
    valores = []
    
    for n in args:
        valores.append(n.get())

    # Defino patrones a validar -> patrones(Concepto, Valor, Fecha, Desscripcion )
    patrones = (re.compile('\w+'), re.compile('\d+'), re.compile('\d+/\d+/\d{4}'), re.compile('\w+'))

    try:
        i = 0
        for patron in patrones:
            if patron.match(valores[i]):
                i += 1
                print(f"Campo {i} OK")
            else:
                raise TypeError(f"Campo {i} inválido")

    except TypeError as mensaje:
        return mensaje
    else:
        return valores